In [1]:
include("src/weather/load_weather.jl")
include("src/route/domain.jl")
include("src/performance/polar.jl")

perf_interp

# sail_route.jl Development

Notebook to enable the development of functions used for sail_route.jl

## Focus

Developing the shortest path routing algorithm.

- [x] Specify the start and end points
- [x] Calculate the analytical solution for the fastest route between two points
- [x] Load control weather scenario.
- [x] Check interpolation from weather scenario
- [x] Generate grid
- [x] Simple cost function
- [x] First for loop
- [x] Generate index array
- [x] Intermediate for loop
- [x] End for loop
- [ ] Shortest path 
- [x] Check that the times work
- [ ] Load current data
- [ ] Optimum path considering the current


The domain is from 0 to 10 in both the x and y directions.

- Start = (0, 5) (x, y)
- Finish = (10, 5) 

In [2]:
# Analytical solution
lon1 = 0.0
lat1 = 5.0
lon2 = 10.0
lat2 = 5.0

d_an, b_an = haversine(lon1, lat1, lon2, lat2)
d_an_str = @sprintf("%0.3f", d_an)
b_an_str = @sprintf("%0.3f", b_an)
println("Analytical distance is $d_an_str nm")
println("Analytical bearing is $b_an_str deg")

In [3]:
# start time 
start = Dates.DateTime(2000, 1, 1, 0, 6, 0)
x_int = 5.0
y_int = 5.0
println("Start time ", start)

# test interpolation of weather scenario
wisi, widi = sample_weather()

w_int = wisi[:interp](time=start, lon_b=x_int, lat_b=y_int)[:data]
@time println(w_int)

Analytical distance is 598.283 nm
Analytical bearing is 89.563 deg
Start time 2000-01-01T00:06:00
10.0
 

In [4]:
# generate grid 
x, y, land = co_ordinates(lon1, lon2, lat1, lat2, 5, 5, 100000)
println(maximum(x), " ", minimum(x))
println(maximum(y), " ", minimum(y))
println(x)

 0.393087 seconds (359.73 k allocations: 18.025 MiB, 2.27% gc time)
8.333369236174814 1.6666307638251738
6.098621432057039 3.9293327250178742


In [5]:
# load performance data 
path = ENV["HOME"]*"/Documents/sail_route.jl/src/data/first40_orgi.csv"
twa, tws, perf = load_file(path)
polar = setup_interpolation(tws, twa, perf)


# cost function
# ws_int, Interpolated wind speed (kts)
# wd_int, Interpolated wind direction (deg)
# d, Distance (nm)
# b, Bearing (nm)
function cost_function(polar, ws_int, wd_int, d, b)
    vs = perf_interp(polar, min_angle(wd_int, b), ws_int)
    return d/vs
end

d, b = haversine(lon1, lat1, lon2, lat2)
time_an = cost_function(polar, 10.0, 10.0, d, b)
println(typeof(time_an))
println("Analytical time is $time_an hrs")

function convert_time(old_time)
    """Convert hours in float to hours and minutes."""
    h = floor(old_time)
    m = floor((old_time - h)*60)
    return Dates.Hour(h)+Dates.Minute(m)
end

[1.66663 1.66663 1.66663 1.66663 1.66663; 3.3333 3.3333 3.3333 3.3333 3.3333; 5.0 5.0 5.0 5.0 5.0; 6.6667 6.6667 6.6667 6.6667 6.6667; 8.33337 8.33337 8.33337 8.33337 8.33337]


convert_time (generic function with 1 method)

Float64
Analytical time is 76.9907233253178 hrs


In [6]:
@time begin

# print co-ordinates
x, y, land = co_ordinates(lon1, lon2, lat1, lat2, 40, 40, 1000)
empty = zeros(x)
earliest_times = fill!(empty, Inf)
prev_node = zeros(x)
node_indices = reshape(1:length(x), size(x)) # look at existing method for calculating indices
arrival_time = Inf

for idx in 1:size(x)[2]
    d, b = haversine(lon1, lat1, x[1, idx], y[1, idx])
    ws_int = wisi[:interp](time=start, lon_b=x[1, idx], lat_b=y[1, idx])[:data]
    wd_int = widi[:interp](time=start, lon_b=x[1, idx], lat_b=y[1, idx])[:data]
    # include current here
    earliest_times[1, idx] = cost_function(polar, ws_int[1], wd_int[1], d, b)
end
 

for idy in 1:size(x)[1]-1
    for idx in 1:size(x)[2]
        t = start + convert_time(earliest_times[idy, idx])
        d, b = haversine(x[idy, idx], y[idy, idx], x[idy+1, idx], y[idy+1, idx])
        ws_int = wisi[:interp](time=t, lon_b=x[idy, idx], lat_b=y[idy, idx])[:data]
        wd_int = widi[:interp](time=t, lon_b=x[idy, idx], lat_b=y[idy, idx])[:data]
        tt = earliest_times[idy, idx] + cost_function(polar, ws_int[1], wd_int[1], d, b) 
        if earliest_times[idy+1, idx] > tt
            earliest_times[idy+1, idx] = tt
        end
    end
end

for idx in 1:size(x)[2]
    d, b = haversine(x[end, idx], y[end, idx], lon2, lat2)
    t = start + convert_time(earliest_times[end, idx])
    ws_int = wisi[:interp](time=start, lon_b=x[end, idx], lat_b=y[end, idx])[:data]
    wd_int = widi[:interp](time=start, lon_b=x[end, idx], lat_b=y[end, idx])[:data]
    tt = earliest_times[end, idx] + cost_function(polar, ws_int[1], wd_int[1], d, b) 
    if arrival_time > tt
        arrival_time = tt
    end
end
    
end
print(arrival_time)

In [7]:


function h(cudi, cusp, bearing)
   cusp*sin(deg2rad(cudi-bearing))
end


function current(polar, cudi, cusp, widi, wisp, bearing, heading)
    vs = perf_interp(polar, min_angle(widi, heading), wisp)
    return (acos(h(cudi, cusp, bearing)/vs)*180/π - bearing)
end

current (generic function with 1 method)

 28.216652 seconds (671.34 k allocations: 26.289 MiB, 0.05% gc time)
75.6450020999251

In [16]:
# iterate heading values until they converge
# current(polar, cudi, cusp, widi, wisp, bearing, heading)
cudi = 180.0
cusp = 0.0
widi = 0.0
wisp = 10.0
bearing = 90.0
h1 = 0.0
h2 = current(polar, cudi, cusp, widi, wisp, bearing, h1)
@time while h2 - h1 > 0.1
    h1 = h2
    h2 = current(polar, cudi, cusp, widi, wisp, bearing, h1)
end
println(h2)
println(bearing + h2)


function correct_speed(polar, cudi, cusp, widi, wisp, bearing)
    """Identify corrected speed for routing."""
    h1 = 0.0
    h2 = current(polar, cudi, cusp, widi, wisp, bearing, h1)
    while h2 - h1 > 0.1
            h1 = h2
            h2 = current(polar, cudi, cusp, widi, wisp, bearing, h1)
    end
    bearing = bearing + h2
    vs = perf_interp(polar, min_angle(widi, bearing), wisp)
    println(vs)
    return vs + cusp
end

correct_speed(polar, cudi, cusp, widi, wisp, bearing)

7.91

  0.000011 seconds (1 allocation: 16 bytes)
0.0
90.0
7.91
